In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
! pwd
! whoami

/home/reflex/refleX/notebooks
reflex


In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import joblib
import sys

In [4]:
sys.path.insert(0, '/home/reflex/refleX/src/')

In [28]:
polarizing_method = 'max'
IMAGE_DIRECTORY = '../metadata/6K/original512/'
CENTER_FILE = '../metadata/6K/csv/centers.csv'
POLAR_DIRECTORY = f'../metadata/6K/polar512/{polarizing_method}/'
os.makedirs(POLAR_DIRECTORY)
#AUGMENTED_DIRECTORY = '../metadata/labeled/augmented512/'
#AUGMENTED_POLAR_DIRECTORY = '../metadata/labeled/augmented_polar512/'

### Augmentacja obrazów niepolarnych

In [9]:
def augment(img):
    t0 = [
        img,
        np.flip(img, 0)
    ]
    
    t1 = []
    for i in t0:
        for k in range(4):
            t1.append(np.rot90(i, k))
    
    t2 = []
    for i in t1:
        t2.append(i+1)
        t2.append(i)
        t2.append(i-1)
        
    return t2

In [23]:
def _augment_directory():
    filenames = os.listdir(IMAGE_DIRECTORY)
    print(len(filenames))
    for img_filename in filenames:
        original = cv2.imread(f'{IMAGE_DIRECTORY}{img_filename}', 1)
        transformations = augment(original)
        for i, t in enumerate(transformations):
            cv2.imwrite(f'{AUGMENTED_DIRECTORY}{img_filename[:-4]}_{i}.png', t)

2228


### Transformacja obrazów do współrzędnych polarnych

https://docs.opencv.org/3.4.3/da/d54/group__imgproc__transform.html

enum  	cv::InterpolationFlags { 
  cv::INTER_NEAREST = 0, 
  cv::INTER_LINEAR = 1, 
  cv::INTER_CUBIC = 2, 
  cv::INTER_AREA = 3, 
  cv::INTER_LANCZOS4 = 4, 
  cv::INTER_LINEAR_EXACT = 5, 
  cv::INTER_MAX = 7, 
  cv::WARP_FILL_OUTLIERS = 8, 
  cv::WARP_INVERSE_MAP = 16 
}
 	interpolation algorithm More...
 
enum  	cv::InterpolationMasks { 
  cv::INTER_BITS = 5, 
  cv::INTER_BITS2 = INTER_BITS * 2, 
  cv::INTER_TAB_SIZE = 1 << INTER_BITS, 
  cv::INTER_TAB_SIZE2 = INTER_TAB_SIZE * INTER_TAB_SIZE 
}

In [29]:
center_df = pd.read_csv(CENTER_FILE, index_col=0) # column order is x, y, (0,0) is NW
print(sorted(os.listdir(IMAGE_DIRECTORY))[:5])
center_df.sort_index().iloc[:5, :]

['000001d8_x0001.png', '011884_KH.4.5_1_0001.png', '012132_NY32-6_1_0001.png', '013209_A8_3_3_0001.png', '013574_P6_9_1_0001.png']


,x,y,mask_start,mask_end
image,,,,
000001d8_x0001.png,255,255,16.875000,21.093750
011884_KH.4.5_1_0001.png,253,262,267.890625,274.570312
012132_NY32-6_1_0001.png,253,265,269.296875,277.031250
013209_A8_3_3_0001.png,254,259,267.539062,274.570312
013574_P6_9_1_0001.png,253,259,269.648438,276.328125


In [30]:
from polarutils import img2polar, polarize

In [32]:
_ = joblib.Parallel(n_jobs=2, prefer="threads")(
    joblib.delayed(polarize)(img_filename, IMAGE_DIRECTORY, POLAR_DIRECTORY, center_df, polarizing_method) for img_filename in os.listdir(IMAGE_DIRECTORY))
    

In [79]:
orig_df = pd.read_csv('../metadata/csv/cleaned_reflex.csv', index_col=0)
aug_df = pd.DataFrame(columns = orig_df.columns)
j = 0
for index, row in orig_df.iterrows():
    if j%100 == 0:
        print(j)
    j += 1
    _aug_df = pd.DataFrame(columns = orig_df.columns)
    for i in range(24):
        _aug_df.loc[f'{index}_{i}'] = row
    aug_df = aug_df.append(_aug_df)
aug_df

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


,0,1,0.1,1.1,0.2,0.3,0.4
159769_2_0001_0,0,1,0,0,0,0,0
159769_2_0001_1,0,1,0,0,0,0,0
159769_2_0001_2,0,1,0,0,0,0,0
159769_2_0001_3,0,1,0,0,0,0,0
159769_2_0001_4,0,1,0,0,0,0,0
159769_2_0001_5,0,1,0,0,0,0,0
159769_2_0001_6,0,1,0,0,0,0,0
159769_2_0001_7,0,1,0,0,0,0,0
159769_2_0001_8,0,1,0,0,0,0,0
159769_2_0001_9,0,1,0,0,0,0,0


In [85]:
aug_df.head()

,0,1,0.1,1.1,0.2,0.3,0.4
159769_2_0001_0,0,1,0,0,0,0,0
159769_2_0001_1,0,1,0,0,0,0,0
159769_2_0001_2,0,1,0,0,0,0,0
159769_2_0001_3,0,1,0,0,0,0,0
159769_2_0001_4,0,1,0,0,0,0,0


In [86]:
print(aug_df.shape)

(53448, 7)


In [87]:
aug_df.to_csv('../metadata/csv/augmented.csv')

In [24]:
full_df = pd.read_csv('../metadata/csv/augmented.csv', index_col=0)
print(len(full_df))
sample_df = pd.DataFrame(columns=full_df.columns)
print(len(sample_df))

53448
0


In [25]:
df_indeces = sorted(full_df.index.tolist())
file_names = os.listdir('../metadata/labeled/augmented_polar512_sample/')
file_names = sorted(list(map(lambda x: x[:x.rfind('.')], file_names)))
i = 0
j = 0
while i<len(df_indeces) and j<len(file_names):
    if i%10000==0:
        print(i)
    if df_indeces[i] < file_names[j]:
        i += 1
    elif df_indeces[i] > file_names[j]:
        j += 1
    else:
        sample_df = sample_df.append(full_df.loc[df_indeces[i]])
        i += 1
        j += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000


In [26]:
print(len(sample_df))
sample_df.head()

13359


,0,1,0.1,1.1,0.2,0.3,0.4
000001d8_x0001_0,0,0,1,0,0,0,0
000001d8_x0001_1,0,0,1,0,0,0,0
000001d8_x0001_11,0,0,1,0,0,0,0
000001d8_x0001_16,0,0,1,0,0,0,0
000001d8_x0001_2,0,0,1,0,0,0,0


In [27]:
sample_df.to_csv('../metadata/csv/augmented_sample.csv')

In [ ]:
def get_center(img):
    flipped = img.copy()
    cv2.flip(flipped, -1)
    #https://docs.opencv.org/3.4/df/dfb/group__imgproc__object.html#ga3a7850640f1fe1f58fe91a2d7583695d
    ans = cv2.matchTemplate(img, flipped, cv2.TM_CCORR_NORMED)
    print(ans) # TODO ans will be 1x1 if dims of img and template are same
    plt.show()
    return cv2.minMaxLoc(ans)

_fn = os.listdir(IMAGE_DIRECTORY)[0]
_img = cv2.imread(IMAGE_DIRECTORY+_fn, 1)
print(get_center(_img))